In [1]:
suppressPackageStartupMessages(library('tidyverse'))
suppressPackageStartupMessages(library('tidylog'))
suppressPackageStartupMessages(library('sf'))
suppressPackageStartupMessages(library('googlePolylines'))

In [2]:
# Aplicar funcoes em paralelo
library('future.apply')
# Checando: Jupyter suporta multicore?
future::supportsMulticore()

Loading required package: future



[1] TRUE

In [3]:
# https://stackoverflow.com/questions/40536067/how-to-adjust-future-global-maxsize
# For 850MB: 850*1024^2 = 891289600
# For +1.3GB: 1500*1024^2 = 1572864000
# options(future.globals.maxSize = 891289600)
options(future.globals.maxSize = 5e6) # 5 MB
options(future.globals.maxSize = 140*1024^2) # 140 MB
parallelly::availableCores()

system 
     8

In [4]:
# Estrutura de pastas
pasta_dados       <- "../../yellow_dados"
pasta_aop_rev     <- sprintf("%s/12_aop_revisitado", pasta_dados)
pasta_aoprv_alter <- sprintf("%s/03_aop_alternatives_2019_2028", pasta_aop_rev)
pasta_ids_aopt_19 <- sprintf("%s/A_2019_osm_way_ids_aop", pasta_aoprv_alter)
# pasta_rts_aopt_19 <- sprintf("%s/B_2019_rotas_modeladas_alternatives", pasta_aoprv_alter)
pasta_ids_aopt_28 <- sprintf("%s/C_2028_osm_way_ids_aop", pasta_aoprv_alter)
# pasta_rts_aopt_28 <- sprintf("%s/D_2028_rotas_modeladas_alternatives", pasta_aoprv_alter)

ano <- '2019'; min_thres <- 40; sec_thres <- min_thres * 60
# ano <- '2028'; min_thres <- 40; sec_thres <- min_thres * 60

if (ano == '2019') {
  pasta_tmp_osmids  <- sprintf("%s/E_%s_osm_way_ids_tmp_%s_min", pasta_aoprv_alter, ano, min_thres)
} else if (ano == '2028') {
  pasta_tmp_osmids  <- sprintf("%s/F_%s_osm_way_ids_tmp_%s_min", pasta_aoprv_alter, ano, min_thres)
}
dir.create(pasta_tmp_osmids, recursive = TRUE, showWarnings = FALSE)
rm(min_thres)

if (ano == '2019') {
  pasta_tmp_divididas <- sprintf("%s/X_%s_tmp_base_dividida", pasta_aoprv_alter, ano)
} else if (ano == '2028') {
  pasta_tmp_divididas <- sprintf("%s/Y_%s_tmp_base_dividida", pasta_aoprv_alter, ano)
}

In [5]:
# ------------------------------------------------------------------------------
# Funções
# ------------------------------------------------------------------------------

# Transforma linha de polyline para dataframe com latlongs
polyline_to_latlong <- function(polyline, trip_id){
  # polyline <- viagem$poly; trip_id <- viagem$trip_id
  this <- as.data.frame(decode(as.character(polyline)))
  this <- this %>% 
    # Formatação dos pontos está fora do lugar: 
    # de -235.641 para -23.5641 - ajeitar
    mutate(trip_id  = trip_id,
           lat = str_replace(lat, '\\.', ''),
           lon = str_replace(lon, '\\.', ''),
           lat = as.double(str_replace(lat, '-23', '-23.')),
           lon = as.double(str_replace(lon, '-46', '-46.')))
  
  return(this)
}


# Transforma dataframe com várias linhas de latlon em sf
df_latlong_to_sf <- function(df, trip_id, st_type = 'LINESTRING'){
  # df <- this
  this <- df %>% 
    # Transformar em sf
    st_as_sf(coords = c("lon", "lat"), crs = 4326) %>% 
    # Transformar pontos em linha - ver possíveis erros em
    # https://github.com/r-spatial/sf/issues/321
    # # Modo 1 - Com st_coordinates, retorna matriz
    # Retrieve coordinates in matrix form 
    # st_coordinates() %>%
    # st_linestring()
    # Modo 2 - Com summarize, retorna sf
    # Aqui, o summarize pode ser qualquer coisa, o 
    # importante é o 'do_union=FALSE'
    group_by(trip_id) %>% 
    summarize(m = n(), do_union = FALSE) %>% 
    select(-m) %>% 
    st_cast(st_type)
  
  return(this)
}

# Insere as distâncias calculadas em vias comuns ou infra cicloviária no df original
dist_ciclo_sep_osmids <- function(trip_id_alt) {
  # trip_id_alt <- rotas_ciclo$alt_id[1]
  # 89a81044d93ffff-89a81046d53ffff-1
  # trip_id_alt <- '000ed3-001d3b-1'
  
  line    <- rotas_ciclo %>% filter(alt_id == trip_id_alt)
  line_id <- str_sub(line$alt_id, 1, 6)
  # line$poly
  
  # Puxar osm_ids da rota modelada
  if (ano == '2019') {
    trip_osm_ids <- list.files(pasta_ids_aopt_19, pattern = line_id, recursive = FALSE, full.names = TRUE) 
  } else if (ano == '2028') {
    trip_osm_ids <- list.files(pasta_ids_aopt_28, pattern = line_id, recursive = FALSE, full.names = TRUE) 
  }
  
  trip_osm_ids <- read_delim(trip_osm_ids, delim = ';', col_types = 'ciic')
  
  # Criar um id que contemple o número da rota alternativa
  trip_osm_ids <- trip_osm_ids %>% mutate(alt_id = str_c(hex_id, alt, sep = '-'))
  
  # Isolar a viagem de interesse
  trip_osm_ids <- trip_osm_ids %>% filter(alt_id == trip_id_alt)
  
  # Remover osm_ids repetidos - no momento do buffer com o shapefile da rota,
  # o intersection vai pegar todos os trechos que os osm_ids aparecem e vai
  # somar essas distâncias. Osm_ids repetidos são quando a rota entra, sai e
  # entra de novo em um mesmo osm_id
  trip_osm_ids <- trip_osm_ids %>% distinct(osm_way_id)
  # trip_way_ids <- paste0("'", trip_osm_ids$osm_way_id, "'", collapse = ",")
  
  # Juntar infraestrutura cicloviária do ano de referência
  trip_osm_ids <- 
    trip_osm_ids %>% 
    left_join(infra_ciclo, by = c('osm_way_id' = 'osm_id')) %>% 
    mutate(infra_ciclo = ifelse(is.na(infra_ciclo), 'via_comum', infra_ciclo))
  
  # Essa rota passa por estrutura cicloviária?
  qtd_infraciclo <- trip_osm_ids %>% filter(str_detect(infra_ciclo, 'ciclo')) %>% nrow()
  
  # # Depurar aviso de...
  # # In left_join(., viario_rota_cropped, by = c(osm_way_id = "osm_id")) :
  # # Detected an unexpected many-to-many relationship between `x` and `y`.
  # # ℹ Row 1 of `x` matches multiple rows in `y`.
  # # ℹ Row 32 of `y` matches multiple rows in `x`.
  # osm_ids_repetidos <- trip_osm_ids %>% group_by(osm_way_id) %>% tally() %>% filter(n > 1) %>% nrow()
  # if (osm_ids_repetidos > 0 & qtd_infraciclo > 0) { print(trip_id_alt) }
  # # trip_osm_ids %>% filter(osm_way_id == '928656996')
  
  if (qtd_infraciclo > 0) {
    # Se passa, calcular as extensões por estrutura cicloviária
    
    # ----------------------------------------------------------------------------
    # Calcular extensões percorridas em vias comuns e infra cicloviária
    # ----------------------------------------------------------------------------
    
    # Filtrar osm_ids da rota modelada do viário completo de SP
    viario_rota <- viario_sp %>% filter(osm_id %in% trip_osm_ids$osm_way_id)
    # mapview(viario_rota)
    
    # Transformar o polyline da rota modelada em shapefile
    shape_rota <- polyline_to_latlong(line$poly, trip_id_alt)
    shape_rota <- df_latlong_to_sf(shape_rota, trip_id_alt)
    shape_rota <- shape_rota %>% st_transform(31983) %>% mutate(dist = round(st_length(.), 4))
    # mapview(shape_rota)
    
    # A partir de um pequeno buffer criado no polyline da rota modelada, fazer uma 
    # interseção nos osm_ids originais - isso porque os osm_ids podem ter várias 
    # quadras e o segmento percorrido ser só um trechinho dele
    buffer_rota <- st_buffer(shape_rota, 2)
    viario_rota_cropped <- suppressWarnings(st_intersection(viario_rota, buffer_rota))
    # mapview(viario_rota_cropped) + mapview(buffer_rota)
    
    # Recaulcular as extensões dos arcos (no caso, as extensões percorridas dentro
    # daquele osm_id), transformar em dataframe e isolar colunas de interesse
    viario_rota_cropped <- viario_rota_cropped %>% mutate(new_ext = as.double(st_length(.)), .after = 'length_m')
    viario_rota_cropped <- viario_rota_cropped %>% st_drop_geometry() %>% select(osm_id, new_ext)
    
    # Juntar com dados da infraestrutura cicloviária
    trip_osm_ids <- trip_osm_ids %>% left_join(viario_rota_cropped, by = c('osm_way_id' = 'osm_id'))
    
    # Fator de ajuste para as distâncias - vamos aplicar um proporcional geral a partir
    # da diferença entre a extensão total da rota modelada e a calculada agora
    fator_correcao <- line$distance / sum(viario_rota_cropped$new_ext)
    trip_osm_ids    <- trip_osm_ids %>% mutate(ext_rev = new_ext * fator_correcao)
    
    
    # Exportar osm_ids com infra cicloviária e extensões percorridas
    ids_out <- trip_osm_ids %>% filter(infra_ciclo != 'via_comum') %>% mutate(hex_id_alt = trip_id_alt, 
                                                                              .before = 'osm_way_id')
    # if (file.exists(ids_file)) {
    #   write_delim(ids_out, ids_file, delim = ';', append = TRUE)
    # } else {
    #   write_delim(ids_out, ids_file, delim = ';', append = FALSE)
    # }
    
    out_ids_tmp_file <- sprintf('%s/%s_%s_tmp_osmids.csv', pasta_tmp_osmids, ano, line_id)
    if (file.exists(out_ids_tmp_file)) {
      write_delim(ids_out, out_ids_tmp_file, delim = ';', append = TRUE)
    } else {
      write_delim(ids_out, out_ids_tmp_file, delim = ';', append = FALSE)
    }
    
    
    # Agrupar extensões por tipo de viário percorrido
    trip_osm_ids <- 
      trip_osm_ids %>% 
      group_by(infra_ciclo) %>% 
      summarise(ext = sum(ext_rev)) %>% 
      ungroup() %>% 
      mutate(hex_id_alt = trip_id_alt) %>% 
      pivot_wider(id_cols = hex_id_alt,
                  names_from = 'infra_ciclo',
                  values_from = ext)
    
    # Checar se todas as colunas de tipo de viário estão como colunas - se não, inserir
    for (i in c('ciclo_expressa', 'ciclo_comum', 'ciclofaixa', 'via_comum')) {
      if (!i %in% names(trip_osm_ids)) {
        # Inserir nova coluna como NA (NA_real_, NA_character_)
        trip_osm_ids <- trip_osm_ids %>% mutate(!!i := 0)
      }
      
    }
    
    # Somar extensões percorridas em infra cicloviária
    trip_osm_ids <- trip_osm_ids %>% mutate(infra_ciclo = ciclo_expressa + ciclo_comum + ciclofaixa,
                                            .after = 'via_comum')
    
  } else {
    # Se não passa, toda a extensão dela vai ser por vias comuns
    trip_osm_ids <- data.frame(hex_id_alt     = trip_id_alt,
                               via_comum      = line$distance, 
                               infra_ciclo    = 0, 
                               ciclo_expressa = 0, 
                               ciclo_comum    = 0, 
                               ciclofaixa     = 0)
  }
  
  # Juntar todas as infos ao dataframe original e reordenar colunas
  trip_out <- 
    line %>% 
    left_join(trip_osm_ids, by = c('alt_id' = 'hex_id_alt')) %>% 
    relocate(c(via_comum, infra_ciclo, ciclo_expressa, ciclo_comum, ciclofaixa), 
             .before = 'poly')
  
  # Reconstituir hex_ids originais e coluna com a numeração da rota alternativa
  trip_out <- 
    trip_out %>% 
    mutate(alt_id = str_replace(alt_id, 
                                '^([a-z0-9]{6})-([a-z0-9]{6})-([0-9])', 
                                '89a81\\1ffff-89a81\\2ffff_\\3'),
           .before = 'alt_id') %>% 
    separate(alt_id, into = c('hex_id', 'alt'), sep = '_', remove = TRUE)
  
  
  # Gravar resultados
  if (file.exists(out_file)) {
    write_delim(trip_out, out_file, delim = ';', append = TRUE)
  } else {
    write_delim(trip_out, out_file, delim = ';', append = FALSE)
  }
  
  # Gravar id no arquivo somente para ids já processados
  this <- data.frame(id = trip_id_alt)
  if (file.exists(out_file_ids)) {
    write_delim(this, out_file_ids, delim = ';', append = TRUE)
  } else {
    write_delim(this, out_file_ids, delim = ';', append = FALSE)
  }
  
  
}

In [6]:
# ------------------------------------------------------------------------------
# Tratamento - rotas que passaram por alguma infra cicloviária
# ------------------------------------------------------------------------------

# Definir arquivos de saída
out_file     <- sprintf('%s/tmp_ttmatrix_%s_infraciclo.csv', pasta_aoprv_alter, ano)
out_file_ids <- sprintf('%s/tmp_ttmatrix_%s_infraciclo_ids.csv', pasta_aoprv_alter, ano)

# Abrir cópia do viário de SP com osm_ids
viario_sp <- read_sf(sprintf('%s/tmp_sao_paulo_osm_filtrado.gpkg', pasta_aoprv_alter))
head(viario_sp, 2)

# Abrir infra cicloviária
infra_ciclo <- sprintf('%s/tmp_infra_ciclo_%s.csv', pasta_aoprv_alter, ano)
infra_ciclo <- read_delim(infra_ciclo, delim = ';', col_types = 'cc')
head(infra_ciclo, 2)

Warning message in fun(libname, pkgname):
“rgeos: versions of GEOS runtime 3.9.1-CAPI-1.14.2
and GEOS at installation 3.9.1dev-CAPI-1.14.1differ”
Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



osm_id,length_m,geom
<chr>,<dbl>,<LINESTRING [m]>
1000285228,416.6857,LINESTRING (325675.3 737227...
1000285229,71.3783,LINESTRING (325658.4 737217...


osm_id,infra_ciclo
<chr>,<chr>
38937645,ciclo_expressa
51392253,ciclo_expressa


In [7]:
detach("package:tidylog")

In [8]:
# Arquivos a processar
arqs <- list.files(pasta_tmp_divididas, pattern = '^tmp_base_dividida_20[0-9]{2}_[0-9]{3}.csv', full.names = TRUE)
head(arqs)

[1] "../../yellow_dados/12_aop_revisitado/03_aop_alternatives_2019_2028/X_2019_tmp_base_dividida/tmp_base_dividida_2019_003.csv"
[2] "../../yellow_dados/12_aop_revisitado/03_aop_alternatives_2019_2028/X_2019_tmp_base_dividida/tmp_base_dividida_2019_004.csv"
[3] "../../yellow_dados/12_aop_revisitado/03_aop_alternatives_2019_2028/X_2019_tmp_base_dividida/tmp_base_dividida_2019_005.csv"
[4] "../../yellow_dados/12_aop_revisitado/03_aop_alternatives_2019_2028/X_2019_tmp_base_dividida/tmp_base_dividida_2019_006.csv"
[5] "../../yellow_dados/12_aop_revisitado/03_aop_alternatives_2019_2028/X_2019_tmp_base_dividida/tmp_base_dividida_2019_007.csv"
[6] "../../yellow_dados/12_aop_revisitado/03_aop_alternatives_2019_2028/X_2019_tmp_base_dividida/tmp_base_dividida_2019_008.csv"

In [9]:
for (arq in arqs) {
  # arq <- arqs[1]
  print(arq)
  
  # Abrir base de rotas a serem processadas
  rotas_ciclo <- read_delim(arq, delim = ';', col_types = 'cddddc')
  
  # Checar quais resultados já foram rodados - abrir lista, puxar ids e remover
  # do dataframe rotas_ciclo se houver  
  if (file.exists(out_file)) {
    arqs_resultados <- read_delim(out_file_ids, delim = ';', col_types = "c")
    rotas_ciclo <- rotas_ciclo %>% filter(!alt_id %in% arqs_resultados$id)
    rm(arqs_resultados)
    print(nrow(rotas_ciclo))
  }
  
  # Se arquivo já foi completamente processado, removê-lo e passar para o seguinte
  if (nrow(rotas_ciclo) == 0) { 
    file.remove(arq) 
    next
  }
  
  # Limpar memória
  gc(T)
  
  # Processar rotas com infraciclo - melhor rodar no Jupyter;
  # se forem só as rotas originais, é ok rodar no RStudio
  # for (id in rotas_ciclo$alt_id) { dist_ciclo_sep_osmids(id) }
  # Rodar função para todos os arquivos- multi thread (Jupyter)
  (start = Sys.time())
  future::plan(future::multicore)
  invisible(future.apply::future_lapply(X   = rotas_ciclo$alt_id,
                                        FUN = dist_ciclo_sep_osmids,
                                        future.seed = TRUE))
  Sys.time()
  Sys.time() - start
  
  # Remover arquivo inteiramente processado
  file.remove(arq) 
  
  # Limpar memória
  rm(rotas_ciclo)  
  gc(T)
  
}